In [5]:
import sys
sys.path.insert(1,'/home/thomas/research/projects/a549_40x/src/memes')
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import pickle

from matplotlib import animation
from matplotlib import pyplot as plt

import os
from os import listdir


from skimage.measure import regionprops
from skimage.morphology import opening, closing, remove_small_holes, remove_small_objects, disk
from skimage.io import imread

from skimage.filters.rank import otsu
from traj_class import fluor_single_cell_traj

from traj_tools import generate_single_cell_img,find_mother,find_mother_label,find_offspring,parse,count_num_fluor_pca_cord

In [6]:
def generate_fluor_long_traj(df,cells,am_record,traj_df,traj,fluor_name,feature_list,fluor_feature_name):
    
    haralick_labels = ["Angular Second Moment",
                   "Contrast",
                   "Correlation",
                   "Sum of Squares: Variance",
                   "Inverse Difference Moment",
                   "Sum Average",
                   "Sum Variance",
                   "Sum Entropy",
                   "Entropy",
                   "Difference Variance",
                   "Difference Entropy",
                   "Information Measure of Correlation 1",
                   "Information Measure of Correlation 2",
                    "Maximal Correlation Coefficient"]
    
    
    traj_record=pd.DataFrame(traj_df.loc[:,'1':str(time_span)])
    traj_record=traj_record.values
    traj_quan,traj_len=traj_record.shape[0],traj_record.shape[1]
    
    traj_xy=[]
    traj_feature=[]
    traj_contour=[]
    traj_cord=[]
    traj_seri=[]
    traj_am_flag=[]
 
    traj_vim_feature_values=[]
    traj_vim_haralick=[]
    traj_vim_norm_haralick=[]
    traj_vim_pca_cord=[]
    
    traj_pcna_feature_values=[]
    traj_pcna_haralick=[]
    traj_pcna_norm_haralick=[]
    traj_pcna_pca_cord=[]

    num_vim_pca_cord=count_num_fluor_pca_cord(cells,fluor_name[0],fluor_feature_name[:2])
    num_pcna_pca_cord=count_num_fluor_pca_cord(cells,fluor_name[1],fluor_feature_name[2:])
    
    for img_num in range(1,traj_len+1):
        obj_num=traj[img_num-1]
        if obj_num!=-1:
            ind=df.loc[(df['ImageNumber']==img_num)&(df['ObjectNumber']==obj_num)].index.tolist()[0]

            if hasattr(cells[ind],fluor_name[0]+'_feature_values') and \
                hasattr(cells[ind],fluor_name[1]+'_feature_values'):
                
                cell_pcna_feature_values = cells[ind].pcna_feature_values[3]
                num_zero_haralick = sum(x==0 for x in cell_pcna_feature_values)

                if num_zero_haralick != len(cell_pcna_feature_values):
            
                    if hasattr(cells[ind],'cell_contour') and hasattr(cells[ind],'pca_cord'):
                        traj_contour.append(cells[ind].cell_contour.points.flatten())
                        traj_cord.append(cells[ind].pca_cord)
                        traj_seri.append([img_num,obj_num])
                        traj_xy.append([df.loc[ind,'Cell_AreaShape_Center_X'],df.loc[ind,'Cell_AreaShape_Center_Y']])
                        traj_feature.append(df.loc[ind,'Cell_AreaShape_Area':'Cell_AreaShape_Solidity'].values.tolist())

                        if ((am_record['ImageNumber']==img_num)&(am_record['ObjectNumber']==obj_num)).any():
                            am_flag=np.asscalar(am_record.loc[(am_record['ImageNumber']==img_num)&\
                                                              (am_record['ObjectNumber']==obj_num),'am_flag'].values)
                            traj_am_flag.append(am_flag)
                        else:
                            traj_am_flag.append(0)

                        exec('traj_vim_feature_values.append(np.array(cells[ind].'+fluor_name[0]+'_feature_values[:3]))')
                        exec('traj_vim_haralick.append(np.array(cells[ind].'+fluor_name[0]+'_feature_values[3]))')
                        exec('traj_vim_norm_haralick.append(np.array(cells[ind].'+fluor_name[0]+'_feature_values[4]))')
                        exec('traj_vim_pca_cord.append(cells[ind].'+fluor_feature_name[0]+'_pca_cord)')

                        exec('traj_pcna_feature_values.append(np.array(cells[ind].'+fluor_name[1]+'_feature_values[:3]))')
                        exec('traj_pcna_haralick.append(np.array(cells[ind].'+fluor_name[1]+'_feature_values[3]))')
                        exec('traj_pcna_norm_haralick.append(np.array(cells[ind].'+fluor_name[1]+'_feature_values[4]))')
                        exec('traj_pcna_pca_cord.append(cells[ind].'+fluor_feature_name[1]+'_pca_cord)')
    
    traj_xy=np.asarray(traj_xy)
    traj_feature=np.asarray(traj_feature)
    traj_contour=np.asarray(traj_contour)
    traj_cord=np.asarray(traj_cord)
    traj_seri=np.asarray(traj_seri)
    
    traj_am_flag=np.asarray(traj_am_flag)
    
    traj_vim_feature_values=np.asarray(traj_vim_feature_values)
    traj_vim_haralick=np.asarray(traj_vim_haralick)
    traj_vim_norm_haralick=np.asarray(traj_vim_norm_haralick)
    traj_vim_pca_cord=np.asarray(traj_vim_pca_cord)  
    
    traj_pcna_feature_values=np.asarray(traj_pcna_feature_values)
    traj_pcna_haralick=np.asarray(traj_pcna_haralick)
    traj_pcna_norm_haralick=np.asarray(traj_pcna_norm_haralick)
    traj_pcna_pca_cord=np.asarray(traj_pcna_pca_cord)  
   
#     mask=traj_fluor_feature_values[:,0]!=0
    
        
#     abn_inds=find_abnormal_fluor(traj_vim_feature_values[:,0],traj_seri[:,0])
#     if len(abn_inds)>0:
#         traj_vim_feature_values[abn_inds,:]=0
#         traj_vim_haralick[abn_inds,:]=0
#         traj_vim_norm_haralick[abn_inds,:]=0
#         print(traj_vim_pca_cord.shape)
#         traj_vim_pca_cord[abn_inds,:]=0
    if len(traj_pcna_feature_values) > 0:
        return traj_feature,traj_contour,traj_cord,traj_seri,traj_am_flag,\
               [traj_vim_feature_values[:,0],traj_vim_feature_values[:,1],traj_vim_feature_values[:,2],\
               traj_vim_haralick,traj_vim_norm_haralick],traj_vim_pca_cord,\
               [traj_pcna_feature_values[:,0],traj_pcna_feature_values[:,1],traj_pcna_feature_values[:,2],\
               traj_pcna_haralick,traj_pcna_norm_haralick],traj_pcna_pca_cord
    else:
        return traj_feature,traj_contour,traj_cord,traj_seri,traj_am_flag,\
               traj_vim_feature_values,traj_vim_pca_cord,\
               traj_pcna_feature_values,traj_pcna_pca_cord


In [9]:
main_path='C:/Users/14432/OneDrive/Research/Projects/a549_pcna/data/'
drive_path='D:/a549_pcna_10-06-21_144hr_no-treatment/'

output_path=main_path+'out/10-06-21_144hr_no-treatment/'
sct_path=output_path+'sct-02/'
cell_ani_path=output_path+'cell_ani/'
cell_nuc_ani_path=output_path+'cell_nuc_ani/'
if not os.path.exists(sct_path):
    os.makedirs(sct_path)
if not os.path.exists(cell_ani_path):
    os.makedirs(cell_ani_path)
if not os.path.exists(cell_nuc_ani_path):
    os.makedirs(cell_nuc_ani_path)
    
posi_end=2
time_span=1729
traj_len_thres=144

feature_list=['mean_intensity','std_intensity','intensity_range','haralick','norm_haralick']
fluor_feature_name=['vimentin_haralick','pcna_haralick']

In [ ]:
#----------------find long sub traj in a family---------------
for posi in range(1,posi_end+1):
    
    posi_label = 'XY' + str(posi)
    print(posi_label)
    dir_path=output_path+posi_label+'/'
    
    cells_path=dir_path+'cells/'
    fluor_img_path=drive_path+posi_label+'_C2/'
    fluor_img_list=sorted(listdir(fluor_img_path))
    seg_path=output_path+posi_label+'/seg/'
    seg_list=sorted(listdir(seg_path))
 
    df=pd.read_csv(dir_path+'Per_Object_relink.csv')
    am_record=pd.read_csv(dir_path+'am_record.csv')
    traj_df=pd.read_csv(dir_path+'traj_object_num.csv')
    mitosis_df=pd.read_csv(dir_path+'mitosis_record.csv')
    with open (cells_path+'pcna_cells-02', 'rb') as fp:
        cells = pickle.load(fp)

    traj_record=pd.DataFrame(traj_df.loc[:,'1':str(time_span)])
    traj_record=traj_record.values

    traj_quan,traj_len=traj_record.shape[0],traj_record.shape[1]

    for traj_label in range(1,traj_quan+1):
        print('now processing trajectory ' + str(traj_label))
        cur_traj=traj_record[traj_label-1,:]
        
        if len(np.where(cur_traj!=-1)[0]):
        
            family_tree={}
            family_tree=find_offspring(df,mitosis_df,family_tree,traj_label=traj_label)
            print('family',family_tree)
            all_branches=list(list(parse(traj_label, family_tree)))
            print('all branch',all_branches)

            for branch in all_branches:
                branch_end_label=branch[-1]
                branch_end_traj=traj_record[branch_end_label-1,:]
                for sub_label in branch:
                    
                    print('    commencing sub-traj ' + str(sub_label))
                    sub_traj=traj_record[sub_label-1,:]
                    if len(np.where(sub_traj!=-1)[0]):
                        sub_start_t=np.where(sub_traj!=-1)[0][0]+1
                        sub_end_t=np.where(sub_traj!=-1)[0][-1]+1

                        if (sub_end_t-sub_start_t)>=traj_len_thres:

                            print('        traj length threshold approved')
                            divide_points=[]
                            traj_name=str(sub_label)+'_'

                            traj_feature,traj_contour,\
                                traj_cord,traj_seri,traj_am_flag,\
                                traj_vim_feature_values,traj_vim_pca_cord,\
                                traj_pcna_feature_values,traj_pcna_pca_cord=generate_fluor_long_traj(
                                df,cells,am_record,\
                                traj_df,sub_traj,\
                                ['vimentin','pcna'],feature_list,fluor_feature_name)

                            if len(traj_seri) >= 13:
                                print('            traj length seri len satisfactory')
                                traj_sct=fluor_single_cell_traj(traj_seri,traj_contour)
                                traj_sct.set_traj_feature(traj_feature)
                                traj_sct.set_traj_cord(traj_cord)
                                traj_sct.set_traj_divide_points(np.array(divide_points))
                                traj_sct.set_traj_am_flag(traj_am_flag)

                                traj_sct.set_traj_fluor_features('vimentin',feature_list,traj_vim_feature_values)
                                traj_sct.set_traj_fluor_pca_cord(fluor_feature_name[0],traj_vim_pca_cord)

                                traj_sct.set_traj_fluor_features('pcna',feature_list,traj_pcna_feature_values)
                                traj_sct.set_traj_fluor_pca_cord(fluor_feature_name[1],traj_pcna_pca_cord)

                                with open(sct_path+'sub_'+posi_label+'_'+traj_name, 'wb') as fp:
                                    pickle.dump(traj_sct, fp)

                                board_size=250

                                fig=plt.figure()
                                ax = fig.add_subplot(111)
                                ims=[]
                                for img_num,obj_num in traj_seri[traj_pcna_feature_values[0]>0]:
                                    img_num=img_num.astype(np.int)
                                    obj_num=obj_num.astype(np.int)
                                    if obj_num!=-1:
                                        img=imread(fluor_img_path+fluor_img_list[img_num-1])
                                        seg_img=imread(seg_path+seg_list[img_num-1])
                                        crop_cell,crop_cell_env=generate_single_cell_img(img,seg_img,img_num,obj_num)
                                        x_c,y_c=crop_cell.shape[0]//2,crop_cell.shape[1]//2
                                        x_l,y_l=board_size//2-x_c,board_size//2-y_c
                                        cell_on_board=np.zeros((board_size,board_size))
                                        cell_on_board[x_l:x_l+crop_cell.shape[0],y_l:y_l+crop_cell.shape[1]]=crop_cell
                                        plt.axis('off')
                                        im=plt.imshow(cell_on_board,animated=True)
                                        t = ax.annotate('t = ' + str(img_num),(160,1))
                                        o = ax.annotate('o = ' + str(obj_num),(210,1))
                                        ims.append([im,t,o])

                                ani = animation.ArtistAnimation(fig, ims)
                                Writer = animation.writers['ffmpeg']
                                writer = Writer(fps=3, metadata=dict(artist='Thomas'), bitrate=-1)
                                ani.save(cell_ani_path+'sub_pcna_'+posi_label+'_'+traj_name+'.mp4',writer)

                                fig=plt.figure()
                                ax = fig.add_subplot(111)
                                ims=[]
                                for img_num,obj_num in traj_seri[traj_pcna_feature_values[0]>0]:

                                    img_num=img_num.astype(np.int)
                                    obj_num=obj_num.astype(np.int)

                                    img=imread(fluor_img_path+fluor_img_list[img_num-1])
                                    seg=imread(seg_path+seg_list[img_num-1])

                                    if obj_num!=-1:

                                        rps=regionprops(seg)
                                        candi_r=[r for r in rps if r.label==obj_num][0]
                                        candi_box=candi_r.bbox

                                        cell_mask = (seg == obj_num)
                                        region_cell_mask = cell_mask[candi_box[0]:candi_box[2], candi_box[1]:candi_box[3]]
                                        crop_img = img[candi_box[0]:candi_box[2], candi_box[1]:candi_box[3]]
                                        cell_img = (img*cell_mask)[candi_box[0]:candi_box[2],candi_box[1]:candi_box[3]]

                                        nuc_mask=cell_img>otsu(cell_img,selem=disk(max(cell_img.shape)/2), mask=cell_img>0)
                                        nuc_mask=remove_small_objects(opening(nuc_mask),100)
                                        nuc_mask=closing(nuc_mask)
                                        nuc_mask=remove_small_holes(nuc_mask,100)

                                        nuc_img=cell_img*nuc_mask

                                        x_c,y_c=nuc_img.shape[0]//2,nuc_img.shape[1]//2
                                        x_l,y_l=board_size//2-x_c,board_size//2-y_c
                                        cell_on_board=np.zeros((board_size,board_size))
                                        cell_on_board[x_l:x_l+nuc_img.shape[0],y_l:y_l+nuc_img.shape[1]]=nuc_img
                                        plt.axis('off')
                                        im=plt.imshow(cell_on_board,animated=True)
                                        t = ax.annotate('t = ' + str(img_num),(160,1))
                                        o = ax.annotate('o = ' + str(obj_num),(210,1))
                                        ims.append([im,t,o])

                #                 ani = animation.ArtistAnimation(fig, ims)
                                ani = animation.ArtistAnimation(fig, ims)
                                Writer = animation.writers['ffmpeg']
                                writer = Writer(fps=3, metadata=dict(artist='Thomas'), bitrate=-1)
                                ani.save(cell_nuc_ani_path+'sub_pcna_'+posi_label+'_'+traj_name+'.mp4',writer)

In [ ]:
for posi in range(1,posi_end+1):
    posi_label = 'XY' + str(posi)
    print(posi_label)
    
    dir_path=output_path+posi_label+'/'
    fluor_img_path=main_path+'ori/'+posi_label+'_C2/'
    fluor_img_list=sorted(listdir(fluor_img_path))
    seg_path=output_path+posi_label+'/seg/'
    seg_list=sorted(listdir(seg_path))
        
    df=pd.read_csv(dir_path+'Per_Object_relink.csv')
    am_record=pd.read_csv(dir_path+'am_record.csv')
    traj_df=pd.read_csv(dir_path+'traj_object_num.csv')
    mitosis_df=pd.read_csv(dir_path+'mitosis_record.csv')
    with open (dir_path+'cells/'+'fluor_cells', 'rb') as fp:
        cells = pickle.load(fp)

    traj_record=pd.DataFrame(traj_df.loc[:,'1':str(time_span)])
    traj_record=traj_record.values
    traj_quan,traj_len=traj_record.shape[0],traj_record.shape[1]

    for traj_label in range(1,traj_quan+1):

        family_tree={}
        family_tree=find_offspring(df,mitosis_df,family_tree,traj_label=traj_label)
        print('family',family_tree)
        all_branches=list(list(parse(traj_label, family_tree)))
        print('all branch',all_branches)

        for branch in all_branches:   
            branch_end_label=branch[-1]
            branch_end_traj=traj_record[branch_end_label-1,:]
            branch_end_t=np.where(branch_end_traj!=-1)[0][-1]+1

            long_traj=-1*np.ones((time_span,))
            divide_points=[]
            traj_name=''
            for sub_label in branch:
                sub_traj=traj_record[sub_label-1,:]
                mask=sub_traj!=-1
                if (np.where(sub_traj!=-1)[0][-1]+1)!=branch_end_t:
                    divide_points.append(np.where(sub_traj!=-1)[0][-1]+1)
                long_traj[mask]=sub_traj[mask]
                traj_name=traj_name+str(sub_label)+'_' 
            
            daughter_flag=find_mother(mitosis_df,traj_label)
            mother_len = 0
            mother_len_thres = 12
            if daughter_flag==1:
                mother_label=find_mother_label(mitosis_df,traj_label)
                mother_traj=traj_record[mother_label-1,:]
                mother_start_t=np.where(mother_traj!=-1)[0][0]+1
                mother_end_t=np.where(mother_traj!=-1)[0][-1]+1
                mother_len=mother_end_t-mother_start_t
                    
                if mother_len >= mother_len_thres:
                    long_traj[mother_end_t-mother_len_thres:mother_end_t]=\
                        mother_traj[mother_end_t-mother_len_thres:mother_end_t]
                else:
                    mother_mask=mother_traj!=-1
                    long_traj[mother_mask]=mother_traj[mother_mask]
            traj_start_t=np.where(long_traj!=-1)[0][0]+1
                           
            whole_branch=long_traj[traj_start_t-1:branch_end_t]
            long_traj_loss_ratio=whole_branch[whole_branch==-1].shape[0]*1.0/whole_branch.shape[0]
            
            if long_traj_loss_ratio<loss_ratio_thres and (branch_end_t-traj_start_t)>=traj_len_thres:

                traj_feature,traj_contour,\
                    traj_cord,traj_seri,traj_am_flag,\
                    traj_vim_feature_values,traj_vim_pca_cord,\
                    traj_pcna_feature_values,traj_pcna_pca_cord=generate_fluor_long_traj(
                    df,cells,am_record,\
                    traj_df,long_traj,\
                    ['vimentin','pcna'],feature_list,fluor_feature_name)
                if len(traj_seri) >= 25:
                    traj_sct=fluor_single_cell_traj(traj_seri,traj_contour)
                    traj_sct.set_traj_feature(traj_feature)
                    traj_sct.set_traj_cord(traj_cord)
                    traj_sct.set_traj_divide_points(np.array(divide_points))
                    traj_sct.set_traj_am_flag(traj_am_flag)

                    traj_sct.set_traj_fluor_features('vimentin',feature_list,traj_vim_feature_values)
                    traj_sct.set_traj_fluor_pca_cord(fluor_feature_name[0],traj_vim_pca_cord)

                    traj_sct.set_traj_fluor_features('pcna',feature_list,traj_pcna_feature_values)
                    traj_sct.set_traj_fluor_pca_cord(fluor_feature_name[1],traj_pcna_pca_cord)

                    if branch_end_label==traj_label:
                        with open(sct_path+'fluor_sct_'+posi_label+'_'+traj_name+'long_traj', 'wb') as fp:
                            pickle.dump(traj_sct, fp)
                            
                        board_size=250

                        fig=plt.figure()
                        ax = fig.add_subplot(111)
                        ims=[]
                        for img_num,obj_num in traj_seri[traj_pcna_feature_values[0]>0]:
                            img_num=img_num.astype(np.int)
                            obj_num=obj_num.astype(np.int)
                            if obj_num!=-1:
                                img=imread(fluor_img_path+fluor_img_list[img_num-1])
                                seg_img=imread(seg_path+seg_list[img_num-1])
                                crop_cell,crop_cell_env=generate_single_cell_img(img,seg_img,img_num,obj_num)
                                x_c,y_c=crop_cell.shape[0]//2,crop_cell.shape[1]//2
                                x_l,y_l=board_size//2-x_c,board_size//2-y_c
                                cell_on_board=np.zeros((board_size,board_size))
                                cell_on_board[x_l:x_l+crop_cell.shape[0],y_l:y_l+crop_cell.shape[1]]=crop_cell
                                plt.axis('off')
                                im=plt.imshow(cell_on_board,animated=True)
                                t = ax.annotate('t = ' + str(img_num),(160,1))
                                o = ax.annotate('o = ' + str(obj_num),(210,1))
                                ims.append([im,t])

                            ani = animation.ArtistAnimation(fig, ims)
                            ani = animation.ArtistAnimation(fig, ims)
                            Writer = animation.writers['ffmpeg']
                            writer = Writer(fps=3, metadata=dict(artist='Thomas'), bitrate=-1)
                        ani.save(cell_ani_path+'pcna_sct_'+posi_label+'_'+traj_name+'long_traj.mp4',writer)

                        fig=plt.figure()
                        ax = fig.add_subplot(111)
                        ims=[]
                        for img_num,obj_num in traj_seri[traj_pcna_feature_values[0]>0]:

                            img_num=img_num.astype(np.int)
                            obj_num=obj_num.astype(np.int)

                            img=imread(fluor_img_path+fluor_img_list[img_num-1])
                            seg=imread(seg_path+seg_list[img_num-1])

                            if obj_num!=-1:

                                rps=regionprops(seg)
                                candi_r=[r for r in rps if r.label==obj_num][0]
                                candi_box=candi_r.bbox

                                cell_mask = (seg == obj_num)
                                region_cell_mask = cell_mask[candi_box[0]:candi_box[2], candi_box[1]:candi_box[3]]
                                crop_img = img[candi_box[0]:candi_box[2], candi_box[1]:candi_box[3]]
                                cell_img = (img*cell_mask)[candi_box[0]:candi_box[2],candi_box[1]:candi_box[3]]

                                nuc_mask=cell_img>otsu(cell_img,selem=disk(max(cell_img.shape)/2), mask=cell_img>0)
                                nuc_mask=remove_small_objects(opening(nuc_mask),100)
                                nuc_mask=closing(nuc_mask)
                                nuc_mask=remove_small_holes(nuc_mask,100)

                                nuc_img=cell_img*nuc_mask

                                x_c,y_c=nuc_img.shape[0]//2,nuc_img.shape[1]//2
                                x_l,y_l=board_size//2-x_c,board_size//2-y_c
                                cell_on_board=np.zeros((board_size,board_size))
                                cell_on_board[x_l:x_l+nuc_img.shape[0],y_l:y_l+nuc_img.shape[1]]=nuc_img
                                plt.axis('off')
                                im=plt.imshow(cell_on_board,animated=True)
                                t = ax.annotate('t = ' + str(img_num),(160,1))
                                o = ax.annotate('o = ' + str(obj_num),(210,1))
                                ims.append([im,t])

        #                 ani = animation.ArtistAnimation(fig, ims)
                        ani = animation.ArtistAnimation(fig, ims)
                        Writer = animation.writers['ffmpeg']
                        writer = Writer(fps=3, metadata=dict(artist='Thomas'), bitrate=-1)
                        ani.save(cell_nuc_ani_path+'pcna_sct_'+posi_label+'_'+traj_name+'long_traj.mp4',writer)
                        
                    else:
                        with open(sct_path+'indirect_fluor_sct_'+posi_label+'_'+traj_name+'long_traj', 'wb') as fp:
                            pickle.dump(traj_sct, fp)


                        board_size=250

                        fig=plt.figure()
                        ax = fig.add_subplot(111)
                        ims=[]
                        for img_num,obj_num in traj_seri[traj_pcna_feature_values[0]>0]:
                            img_num=img_num.astype(np.int)
                            obj_num=obj_num.astype(np.int)
                            if obj_num!=-1:
                                img=imread(fluor_img_path+fluor_img_list[img_num-1])
                                seg_img=imread(seg_path+seg_list[img_num-1])
                                crop_cell,crop_cell_env=generate_single_cell_img(img,seg_img,img_num,obj_num)
                                x_c,y_c=crop_cell.shape[0]//2,crop_cell.shape[1]//2
                                x_l,y_l=board_size//2-x_c,board_size//2-y_c
                                cell_on_board=np.zeros((board_size,board_size))
                                cell_on_board[x_l:x_l+crop_cell.shape[0],y_l:y_l+crop_cell.shape[1]]=crop_cell
                                plt.axis('off')
                                im=plt.imshow(cell_on_board,animated=True)
                                t = ax.annotate('t = ' + str(img_num),(160,1))
                                o = ax.annotate('o = ' + str(obj_num),(210,1))
                                ims.append([im,t,o])

                            ani = animation.ArtistAnimation(fig, ims)
                            ani = animation.ArtistAnimation(fig, ims)
                            Writer = animation.writers['ffmpeg']
                            writer = Writer(fps=3, metadata=dict(artist='Thomas'), bitrate=-1)
                        ani.save(cell_ani_path+'indirect_pcna_sct_'+posi_label+'_'+traj_name+'long_traj.mp4',writer)

                        fig=plt.figure()
                        ax = fig.add_subplot(111)
                        ims=[]
                        for img_num,obj_num in traj_seri[traj_pcna_feature_values[0]>0]:

                            img_num=img_num.astype(np.int)
                            obj_num=obj_num.astype(np.int)

                            img=imread(fluor_img_path+fluor_img_list[img_num-1])
                            seg=imread(seg_path+seg_list[img_num-1])

                            if obj_num!=-1:

                                rps=regionprops(seg)
                                candi_r=[r for r in rps if r.label==obj_num][0]
                                candi_box=candi_r.bbox

                                cell_mask = (seg == obj_num)
                                region_cell_mask = cell_mask[candi_box[0]:candi_box[2], candi_box[1]:candi_box[3]]
                                crop_img = img[candi_box[0]:candi_box[2], candi_box[1]:candi_box[3]]
                                cell_img = (img*cell_mask)[candi_box[0]:candi_box[2],candi_box[1]:candi_box[3]]

                                nuc_mask=cell_img>otsu(cell_img,selem=disk(max(cell_img.shape)/2), mask=cell_img>0)
                                nuc_mask=remove_small_objects(opening(nuc_mask),100)
                                nuc_mask=closing(nuc_mask)
                                nuc_mask=remove_small_holes(nuc_mask,100)

                                nuc_img=cell_img*nuc_mask

                                x_c,y_c=nuc_img.shape[0]//2,nuc_img.shape[1]//2
                                x_l,y_l=board_size//2-x_c,board_size//2-y_c
                                cell_on_board=np.zeros((board_size,board_size))
                                cell_on_board[x_l:x_l+nuc_img.shape[0],y_l:y_l+nuc_img.shape[1]]=nuc_img
                                plt.axis('off')
                                im=plt.imshow(cell_on_board,animated=True)
                                t = ax.annotate('t = ' + str(img_num),(160,1))
                                o = ax.annotate('o = ' + str(obj_num),(210,1))
                                ims.append([im,t,o])

        #                 ani = animation.ArtistAnimation(fig, ims)
                        ani = animation.ArtistAnimation(fig, ims)
                        Writer = animation.writers['ffmpeg']
                        writer = Writer(fps=3, metadata=dict(artist='Thomas'), bitrate=-1)
                        ani.save(cell_nuc_ani_path+'indirect_pcna_sct_'+posi_label+'_'+traj_name+'long_traj.mp4',writer)